In [1]:
import torch
import pyro
import pandas as pd
import altair as alt
import seaborn as sns
import numpy as np
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.optim import Adam

In [2]:
df = pd.read_hdf("data/watch_minutes.hdf")

In [3]:
df

,user_id,video_id,watched_minutes
0,0,0,1
1,0,39,3
2,0,52,27
3,0,53,85
4,0,62,88
...,...,...,...
29306,12623,116,1
29307,12624,119,32
29308,12625,119,6
29309,12626,53,16


In [4]:
users = df.groupby('user_id', as_index=False).agg({'video_id': len})
users
# sns.kdeplot(users['video_id'])

,user_id,video_id
0,0,38
1,1,2
2,2,5
3,3,3
4,4,3
...,...,...
12623,12623,1
12624,12624,1
12625,12625,1
12626,12626,1


In [5]:
pivot = df.pivot(index="user_id", columns="video_id", values="watched_minutes")

In [6]:
pivot.head()

video_id,0,1,2,3,4,5,6,7,8,9,...,555,556,557,558,559,560,561,562,563,564
user_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,3.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
pd.notna(pivot).sum(axis=1).sort_values(ascending=False)

user_id
10       179
142      101
1442      63
1432      46
1484      44
        ... 
2831       1
7345       1
2832       1
2833       1
12627      1
Length: 12628, dtype: int64

In [8]:
1 - np.sum(np.sum(pd.isna(pivot))) / (pivot.shape[0] * pivot.shape[1])

0.004108162504450008

In [9]:
def split_train_test(data, percent_test=0.1):

    n, m = data.shape
    N = n * m

   
    train = data.copy()
    test = np.ones(data.shape) * np.nan

   
    tosample = np.where(~np.isnan(train)) 
    idx_pairs = list(zip(tosample[0], tosample[1])) 

    test_size = int(len(idx_pairs) * percent_test)  
    train_size = len(idx_pairs) - test_size 

    indices = np.arange(len(idx_pairs))
    sample = np.random.choice(indices, replace=False, size=test_size)

    
    for idx in sample:
        idx_pair = idx_pairs[idx]
        test[idx_pair] = train[idx_pair] 
        train[idx_pair] = np.nan

   
    assert train_size == N - np.isnan(train).sum()
    assert test_size == N - np.isnan(test).sum()

    return train, test

In [10]:
data = pivot.values

In [11]:
train, test = split_train_test(data)

In [12]:
 def model(alpha, dim, n, m, nan_mask, not_na, data):
        """
        Perform matrix factorization
        R = U @ V.T
        """
        alpha_loc = torch.tensor(1 / 25)

        loc_u = pyro.sample(
            "loc_u",
            dist.MultivariateNormal(
                loc=torch.zeros(dim),
                precision_matrix=torch.eye(dim) * alpha_loc,
            ),
        )
        precission_u = pyro.sample(
            "precission_u",
            dist.LKJCorrCholesky(
                d=dim, eta=torch.tensor(alpha)
            ),
        )

        observations_scale = pyro.sample(
            "obs_scale",
            dist.InverseGamma(
                concentration=torch.tensor(1.0),
                rate=torch.tensor(1.0),
            ),
        )

        with pyro.plate("users", n):
            U = pyro.sample(
                "U", dist.MultivariateNormal(loc=loc_u, precision_matrix=precission_u)
            )
        with pyro.plate("content", m):
            V = pyro.sample(
                "V", dist.MultivariateNormal(loc=torch.zeros(dim), precision_matrix=torch.eye(dim))
            )
        with pyro.plate("observations", not_na):
            R = pyro.sample(
                "R",
                dist.Normal(loc=(U @ V.T)[~nan_mask], scale=observations_scale),
                obs=data,
            )


In [13]:
nan_mask = np.isnan(train)
not_na = (~nan_mask).sum()
data = torch.from_numpy(train[~nan_mask])


In [14]:
pyro.clear_param_store()
guide = AutoDiagonalNormal(model)
svi = SVI(model, guide, Adam({"lr": 0.001}), loss=Trace_ELBO())
n, m = train.shape
dim = 5
alpha = 2.0
iterations = 3000
train_loss = []
for i in range(iterations):
    loss = svi.step(alpha, dim, n, m, nan_mask, not_na, data)
    train_loss.append(loss / len(data))
    if i % 100 == 0:
        print("[iteration %04d] loss: %.4f" % (i + 1, loss / len(data)))

[iteration 0001] loss: 644.0718
[iteration 0101] loss: 502.1380
[iteration 0201] loss: 496.0840
[iteration 0301] loss: 346.9953
[iteration 0401] loss: 237.2928
[iteration 0501] loss: 322.2172
[iteration 0601] loss: 270.6103
[iteration 0701] loss: 202.9045
[iteration 0801] loss: 204.7128
[iteration 0901] loss: 160.3064
[iteration 1001] loss: 128.2004
[iteration 1101] loss: 176.9880
[iteration 1201] loss: 104.5985
[iteration 1301] loss: 109.4512
[iteration 1401] loss: 95.3538
[iteration 1501] loss: 98.5455
[iteration 1601] loss: 78.9466
[iteration 1701] loss: 60.2336
[iteration 1801] loss: 95.6775
[iteration 1901] loss: 50.0434
[iteration 2001] loss: 71.7623
[iteration 2101] loss: 57.0525
[iteration 2201] loss: 43.2372
[iteration 2301] loss: 44.0895
[iteration 2401] loss: 35.2519
[iteration 2501] loss: 34.2156
[iteration 2601] loss: 35.2013
[iteration 2701] loss: 29.6166
[iteration 2801] loss: 34.2206
[iteration 2901] loss: 26.6531


In [15]:
V = guide.median()['V']
U = guide.median()['U']

R = U @ V.T

In [16]:
guide.median()['loc_u']

tensor([ 1.2819, -1.4941, -1.4945, -1.2986, -0.6687], grad_fn=<ViewBackward>)

In [17]:
guide.median()['precission_u']

tensor([[ 1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.7657,  0.6432,  0.0000,  0.0000,  0.0000],
        [ 0.5365, -0.6267,  0.5651,  0.0000,  0.0000],
        [ 0.4181, -0.5684, -0.5341,  0.4657,  0.0000],
        [ 0.3020, -0.4969, -0.5084, -0.5383,  0.3372]], grad_fn=<CopySlices>)

In [18]:
U_pooled = dist.MultivariateNormal(loc = guide.median()['loc_u'], precision_matrix=guide.median()['precission_u'])

In [33]:
generated_users = torch.from_numpy(np.array([U_pooled.sample().detach().numpy() for _ in range(1000)]))

In [20]:
generated_users = torch.tensor(np.array(samples))

In [37]:
potential_watched_minutes = (generated_users @ V.T)
potential_watched_minutes.shape

torch.Size([1000, 565])

In [41]:
average_potential_watched_minutes = potential_watched_minutes.mean(axis = 0)

In [42]:
average_potential_watched_minutes.shape

torch.Size([565])

In [ ]:

To wrap things up, probabilistic programming gives us superpowers. We are able to handle potential cases we have not seen before, by generating data from our model, and use those generated samples to make decisions.